In [1]:
!pip install opencv-python opencv-python-headless numpy tensorflow dlib

In [3]:
import cv2
import numpy as np
import time
import csv
import os
from datetime import datetime
from typing import List, Tuple, Optional
from pathlib import Path

class FaceDetector:
    # Constants
    AGE_BUCKETS = ["(0-2)", "(4-6)", "(8-12)", "(15-20)", "(25-32)", "(38-43)", "(48-53)", "(60-100)"]
    GENDER_BUCKETS = ["Male", "Female"]
    CONFIDENCE_THRESHOLD = 0.5
    FACE_BLOB_PARAMS = (1.0, (300, 300), [104, 117, 123], True, False)
    AGE_GENDER_BLOB_PARAMS = (1.0, (227, 227), (78.426, 87.769, 114.896), True, False)

    def __init__(self, model_dir: str = "Models", save_dir: str = "detected_faces", use_gpu: bool = True):
        """
        Initialize the FaceDetector with model paths and configurations.
        Args:
            model_dir: Directory containing the model files
            save_dir: Directory to save detected faces
            use_gpu: Whether to use GPU acceleration if available
        """
        self.model_dir = Path(model_dir)
        self.save_dir = Path(save_dir)
        self.model_paths = {
            'face_model': self.model_dir / "deploy_face.prototxt.txt",
            'face_weights': self.model_dir / "res10_300x300_ssd_iter_140000_fp16.caffemodel",
            'age_model': self.model_dir / "age_net.caffemodel",
            'age_proto': self.model_dir / "deploy_age.prototxt",
            'gender_model': self.model_dir / "gender_net.caffemodel",
            'gender_proto': self.model_dir / "deploy_gender.prototxt.txt"
        }
        
        self._validate_directories()
        self._load_models(use_gpu)
        
        # Cache for resized faces to avoid repeated allocations
        self.face_cache = {}
        self.max_cache_size = 100

    def _validate_directories(self) -> None:
        """Validate and create necessary directories."""
        if not self.model_dir.exists():
            raise FileNotFoundError(f"Model directory not found: {self.model_dir}")
        
        self.save_dir.mkdir(parents=True, exist_ok=True)
        
        for name, path in self.model_paths.items():
            if not path.is_file():
                raise FileNotFoundError(f"Model file not found: {path} ({name})")

    def _load_models(self, use_gpu: bool) -> None:
        """Load all required models with optional GPU acceleration."""
        try:
            self.face_net = cv2.dnn.readNetFromCaffe(
                str(self.model_paths['face_model']), 
                str(self.model_paths['face_weights'])
            )
            self.age_net = cv2.dnn.readNetFromCaffe(
                str(self.model_paths['age_proto']), 
                str(self.model_paths['age_model'])
            )
            self.gender_net = cv2.dnn.readNetFromCaffe(
                str(self.model_paths['gender_proto']), 
                str(self.model_paths['gender_model'])
            )

            if use_gpu:
                try:
                    # Enable GPU acceleration if available
                    self.face_net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
                    self.face_net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
                    self.age_net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
                    self.age_net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
                    self.gender_net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
                    self.gender_net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
                    print("GPU acceleration enabled")
                except Exception as e:
                    print(f"GPU acceleration not available: {e}")

        except Exception as e:
            raise RuntimeError(f"Error loading models: {str(e)}")

    def _get_cached_face(self, face_img: np.ndarray) -> np.ndarray:
        """Get or create resized face image from cache."""
        face_hash = hash(face_img.tobytes())
        if face_hash not in self.face_cache:
            if len(self.face_cache) >= self.max_cache_size:
                self.face_cache.pop(next(iter(self.face_cache)))
            self.face_cache[face_hash] = cv2.resize(face_img, (227, 227))
        return self.face_cache[face_hash]

    def detect_faces(self, frame: np.ndarray) -> List[Tuple[int, int, int, int, float]]:
        """
        Detect faces in the frame and return their coordinates with confidence.
        Returns: List of tuples (x, y, x1, y1, confidence)
        """
        face_blob = cv2.dnn.blobFromImage(frame, *self.FACE_BLOB_PARAMS)
        self.face_net.setInput(face_blob)
        detections = self.face_net.forward()
        
        faces = []
        height, width = frame.shape[:2]
        scale = np.array([width, height, width, height])
        
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > self.CONFIDENCE_THRESHOLD:
                box = detections[0, 0, i, 3:7] * scale
                x, y, x1, y1 = box.astype("int")
                # Ensure coordinates are within frame boundaries
                x, y = max(0, x), max(0, y)
                x1, y1 = min(width, x1), min(height, y1)
                faces.append((x, y, x1, y1, confidence))
        return faces

    def predict_age_gender(self, face_img: np.ndarray) -> Tuple[str, str, float, float]:
        """
        Predict age and gender for a given face image.
        Returns: Tuple of (age_group, gender, age_confidence, gender_confidence)
        """
        try:
            if face_img.size == 0:
                return ("Unknown", "Unknown", 0.0, 0.0)

            face_resized = self._get_cached_face(face_img)
            
            # Single blob creation for both networks
            blob = cv2.dnn.blobFromImage(face_resized, *self.AGE_GENDER_BLOB_PARAMS)
            
            # Age prediction
            self.age_net.setInput(blob)
            age_preds = self.age_net.forward()
            age_idx = np.argmax(age_preds)
            age_conf = float(np.max(age_preds))
            
            # Gender prediction
            self.gender_net.setInput(blob)
            gender_preds = self.gender_net.forward()
            gender_idx = np.argmax(gender_preds)
            gender_conf = float(np.max(gender_preds))
            
            return (
                self.AGE_BUCKETS[age_idx],
                self.GENDER_BUCKETS[gender_idx],
                age_conf,
                gender_conf
            )
        except Exception as e:
            print(f"Error in prediction: {str(e)}")
            return ("Unknown", "Unknown", 0.0, 0.0)

class DetectionLogger:
    def __init__(self, output_file: str = "output_data.csv"):
        """Initialize the detection logger with rotating file capability."""
        self.output_dir = Path("logs")
        self.output_dir.mkdir(exist_ok=True)
        self.base_filename = output_file
        self.current_file = self._get_new_logfile()
        self._initialize_csv()
    
    def _get_new_logfile(self) -> Path:
        """Generate a new logfile name with timestamp."""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        return self.output_dir / f"{timestamp}_{self.base_filename}"

    def _initialize_csv(self) -> None:
        """Initialize the CSV file with headers."""
        with open(self.current_file, mode="w", newline="") as file:
            writer = csv.writer(file)
            writer.writerow([
                "Timestamp", "Age", "Gender", "Position", 
                "Age Confidence", "Gender Confidence",
                "Face Confidence"
            ])

    def log_detection(self, data: tuple) -> None:
        """Log a single detection to the CSV file with error handling."""
        try:
            with open(self.current_file, mode="a", newline="") as file:
                writer = csv.writer(file)
                writer.writerow(data)
        except Exception as e:
            print(f"Error logging detection: {str(e)}")

def main():
    try:
        # Initialize detector and logger
        detector = FaceDetector(use_gpu=True)
        logger = DetectionLogger()

        # Initialize video capture
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            raise RuntimeError("Could not access the camera.")

        # Set optimal camera properties
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
        cap.set(cv2.CAP_PROP_FPS, 30)

        print("Controls:")
        print("  'q' - Quit")
        print("  's' - Save current frame")
        print("  'd' - Toggle detection")
        print("  'f' - Toggle fullscreen")

        detection_enabled = True
        fullscreen = False
        frame_count = 0
        start_time = time.time()
        fps = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                print("Error: Could not read frame.")
                break

            frame_count += 1
            current_time = time.time()
            elapsed_time = current_time - start_time
            
            # Update FPS every half second
            if elapsed_time >= 0.5:
                fps = frame_count / elapsed_time
                frame_count = 0
                start_time = current_time
            
            if detection_enabled:
                # Detect faces and predict attributes
                faces = detector.detect_faces(frame)
                
                for (x, y, x1, y1, face_conf) in faces:
                    face = frame[y:y1, x:x1]
                    if face.size > 0:
                        age, gender, age_conf, gender_conf = detector.predict_age_gender(face)
                        
                        # Draw results with improved visuals
                        cv2.rectangle(frame, (x, y), (x1, y1), (255, 0, 0), 2)
                        
                        # Add background for text
                        label = f"{gender}, {age}"
                        conf_label = f"Conf: {face_conf:.2f}"
                        
                        # Calculate text size for background
                        (w1, h1), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
                        (w2, h2), _ = cv2.getTextSize(conf_label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
                        
                        # Draw semi-transparent background
                        overlay = frame.copy()
                        cv2.rectangle(overlay, (x, y - h1 - 20), (x + max(w1, w2), y), (0, 0, 0), -1)
                        cv2.addWeighted(overlay, 0.6, frame, 0.4, 0, frame)
                        
                        # Draw text
                        cv2.putText(frame, label, (x, y - h1 - 5),
                                  cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                        cv2.putText(frame, conf_label, (x, y - 5),
                                  cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                        
                        # Log detection
                        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                        logger.log_detection((
                            timestamp, age, gender, (x, y, x1, y1),
                            age_conf, gender_conf, face_conf
                        ))

            # Display FPS with background
            fps_label = f"FPS: {fps:.1f}"
            (w, h), _ = cv2.getTextSize(fps_label, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
            cv2.rectangle(frame, (5, 5), (w + 15, h + 15), (0, 0, 0), -1)
            cv2.putText(frame, fps_label, (10, h + 10),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            
            # Display frame
            window_name = "Face Detection"
            if fullscreen:
                cv2.namedWindow(window_name, cv2.WND_PROP_FULLSCREEN)
                cv2.setWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
            else:
                cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
            
            cv2.imshow(window_name, frame)

            # Handle key presses
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            elif key == ord('s'):
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                save_path = Path("saved_frames") / f"frame_{timestamp}.jpg"
                save_path.parent.mkdir(exist_ok=True)
                cv2.imwrite(str(save_path), frame)
                print(f"Frame saved as {save_path}")
            elif key == ord('d'):
                detection_enabled = not detection_enabled
                print(f"Detection {'enabled' if detection_enabled else 'disabled'}")
            elif key == ord('f'):
                fullscreen = not fullscreen

    except Exception as e:
        print(f"Error: {str(e)}")
    
    finally:
        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Error: Model directory not found: Models


UnboundLocalError: cannot access local variable 'cap' where it is not associated with a value